# Введение
Метод главных компонент (principal component analysis, PCA) — это один из методов обучения без учителя, который позволяет сформировать новые признаки, являющиеся линейными комбинациями старых. При этом новые признаки строятся так, чтобы сохранить как можно больше дисперсии в данных. Иными словами, метод главных компонент понижает размерность данных оптимальным с точки зрения сохранения дисперсии способом.

Основным параметром метода главных компонент является количество новых признаков. Как и в большинстве методов машинного обучения, нет четких рекомендаций по поводу выбора значения этого параметров. Один из подходов — выбирать минимальное число компонент, при котором объясняется не менее определенной доли дисперсии (это означает, что в выборке сохраняется данная доля от исходной дисперсии).

В этом задании понадобится измерять схожесть двух наборов величин. Если имеется набор пар измерений (например, одна пара — предсказания двух классификаторов для одного и того же объекта), то охарактеризовать их зависимость друг от друга можно с помощью корреляции Пирсона. Она принимает значения от -1 до 1 и показывает, насколько данные величины линейно зависимы. Если корреляция равна -1 или 1, то величины линейно выражаются друг через друга. Если она равна нулю, то линейная зависимость между величинами отсутствует.

# Данные
В этом задании мы будем работать с данными о стоимостях акций 30 крупнейших компаний США. На основе этих данных можно оценить состояние экономики, например, с помощью индекса Доу-Джонса. Со временем состав компаний, по которым строится индекс, меняется. Для набора данных был взят период с 23.09.2013 по 18.03.2015, в котором набор компаний был фиксирован (подробнее почитать о составе можно по ссылке из материалов).

Одним из существенных недостатков индекса Доу-Джонса является способ его вычисления — при подсчёте индекса цены входящих в него акций складываются, а потом делятся на поправочный коэффициент. В результате, даже если одна компания заметно меньше по капитализации, чем другая, но стоимость одной её акции выше, то она сильнее влияет на индекс. Даже большое процентное изменение цены относительно дешёвой акции может быть нивелировано незначительным в процентном отношении изменением цены более дорогой акции.

# Реализация в sklearn
Метод главных компонент реализован в пакете scikit-learn в модуле decomposition в классе PCA. Основным параметром является количество компонент (n_components). Для обученного преобразования этот класс позволяет вычислять различные характеристики. Например, поле explained_variance_ratio_ содержит процент дисперсии, который объясняет каждая компонента. Поле components_ содержит информацию о том, какой вклад вносят признаки в компоненты. Чтобы применить обученное преобразование к данным, можно воспользоваться методом transform.

Для нахождения коэффициента корреляции Пирсона можно воспользоваться функцией corrcoef из пакета numpy.

1. Загрузите данные close_prices.csv. В этом файле приведены цены акций 30 компаний на закрытии торгов за каждый день периода.

In [1]:
import numpy as np
import pandas as pd

In [2]:
close_prices_data = pd.read_csv('_82b4a9f66c689b3d40dd25ebd761b07f_close_prices.csv')
close_prices_data

,date,AXP,BA,CAT,CSCO,CVX,DD,DIS,GE,GS,...,PFE,PG,T,TRV,UNH,UTX,V,VZ,WMT,XOM
0,2013-09-23,76.440002,117.510002,85.029999,24.270000,125.519997,59.409999,64.750000,24.280001,165.250000,...,28.799999,79.279999,34.220001,86.379997,71.820000,109.419998,196.240005,47.980000,76.419998,87.750000
1,2013-09-24,76.070000,119.000000,85.110001,24.139999,124.489998,59.319997,64.320000,24.320000,162.970001,...,28.709999,78.620003,34.090000,85.870003,72.320000,110.000000,193.339996,47.270000,75.750000,87.360001
2,2013-09-25,75.989998,118.510002,84.500000,24.430000,124.070000,59.319997,64.449997,24.230000,162.309998,...,28.490000,77.720001,34.049999,85.980003,71.980003,109.260002,191.559998,46.950001,74.650002,87.139999
3,2013-09-26,76.320000,119.379997,84.199997,23.770000,123.489998,59.509996,65.239998,24.250000,162.289993,...,28.520000,78.050003,34.230000,85.830002,72.160004,109.660004,193.559998,47.669998,74.620003,87.070000
4,2013-09-27,75.889999,118.739998,83.800003,23.330000,122.639999,59.009995,65.190002,24.049999,159.850006,...,28.879999,77.209999,33.980000,85.410004,71.989998,109.360001,193.050003,47.000000,74.360001,86.900002
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
369,2015-03-12,81.559998,152.039993,80.190002,28.240000,102.419998,80.400001,107.169998,25.400000,189.949997,...,34.000000,82.089996,33.130001,107.830002,114.879997,121.239998,269.579987,48.730000,81.900002,84.220001
370,2015-03-13,80.599998,151.570007,79.230003,27.940001,101.620003,80.499997,106.440002,25.040001,189.339996,...,34.000000,81.830002,32.759998,106.720001,115.250000,118.739998,265.029999,48.840000,81.900002,83.870003
371,2015-03-16,81.500000,153.669998,79.970001,28.299999,103.129997,77.069994,107.370003,25.450001,191.899994,...,34.439999,83.559998,33.060001,108.930000,118.519997,120.070000,269.019989,49.270000,83.290001,84.760002
372,2015-03-17,81.059998,154.509995,78.449997,28.150000,103.169998,74.680001,106.959999,25.309999,190.570007,...,34.169998,82.849998,32.900002,108.750000,117.599998,119.860001,264.500000,48.950001,82.620003,84.080002


2. На загруженных данных обучите преобразование PCA с числом компоненты равным 10. Скольких компонент хватит, чтобы объяснить 90% дисперсии?

In [3]:
close_prices_data = close_prices_data.drop('date', axis=1)
close_prices_data

,AXP,BA,CAT,CSCO,CVX,DD,DIS,GE,GS,HD,...,PFE,PG,T,TRV,UNH,UTX,V,VZ,WMT,XOM
0,76.440002,117.510002,85.029999,24.270000,125.519997,59.409999,64.750000,24.280001,165.250000,75.910004,...,28.799999,79.279999,34.220001,86.379997,71.820000,109.419998,196.240005,47.980000,76.419998,87.750000
1,76.070000,119.000000,85.110001,24.139999,124.489998,59.319997,64.320000,24.320000,162.970001,76.040001,...,28.709999,78.620003,34.090000,85.870003,72.320000,110.000000,193.339996,47.270000,75.750000,87.360001
2,75.989998,118.510002,84.500000,24.430000,124.070000,59.319997,64.449997,24.230000,162.309998,75.519997,...,28.490000,77.720001,34.049999,85.980003,71.980003,109.260002,191.559998,46.950001,74.650002,87.139999
3,76.320000,119.379997,84.199997,23.770000,123.489998,59.509996,65.239998,24.250000,162.289993,76.070000,...,28.520000,78.050003,34.230000,85.830002,72.160004,109.660004,193.559998,47.669998,74.620003,87.070000
4,75.889999,118.739998,83.800003,23.330000,122.639999,59.009995,65.190002,24.049999,159.850006,75.959999,...,28.879999,77.209999,33.980000,85.410004,71.989998,109.360001,193.050003,47.000000,74.360001,86.900002
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
369,81.559998,152.039993,80.190002,28.240000,102.419998,80.400001,107.169998,25.400000,189.949997,116.099998,...,34.000000,82.089996,33.130001,107.830002,114.879997,121.239998,269.579987,48.730000,81.900002,84.220001
370,80.599998,151.570007,79.230003,27.940001,101.620003,80.499997,106.440002,25.040001,189.339996,114.820000,...,34.000000,81.830002,32.759998,106.720001,115.250000,118.739998,265.029999,48.840000,81.900002,83.870003
371,81.500000,153.669998,79.970001,28.299999,103.129997,77.069994,107.370003,25.450001,191.899994,116.500000,...,34.439999,83.559998,33.060001,108.930000,118.519997,120.070000,269.019989,49.270000,83.290001,84.760002
372,81.059998,154.509995,78.449997,28.150000,103.169998,74.680001,106.959999,25.309999,190.570007,115.580002,...,34.169998,82.849998,32.900002,108.750000,117.599998,119.860001,264.500000,48.950001,82.620003,84.080002


In [4]:
from sklearn.decomposition import PCA

In [5]:
pca = PCA(n_components=10)

In [6]:
pca.fit(close_prices_data)

PCA(copy=True, iterated_power='auto', n_components=10, random_state=None,
    svd_solver='auto', tol=0.0, whiten=False)

In [7]:
t = pca.transform(close_prices_data)
t

array([[-50.90240358,  17.63167724,   7.7360209 , ...,   3.55657041,
         -5.82197358,  -1.72604005],
       [-52.84690919,  19.14690749,   7.27254551, ...,   3.43259929,
         -5.63318106,  -2.0122316 ],
       [-54.61443917,  19.92735811,   8.10995831, ...,   4.17520282,
         -5.38828678,  -2.75269933],
       ...,
       [ 83.79819948,  16.95241334,  -8.25265611, ...,   2.92390381,
         -4.59543822,  -2.99033503],
       [ 79.19578947,  19.14436008,  -6.93262976, ...,   3.40475421,
         -3.66793327,  -3.57607629],
       [ 82.96885933,  13.13513974,  -9.07644299, ...,   3.75100423,
         -4.79172198,  -3.03918865]])

In [8]:
sum(pca.explained_variance_ratio_[:3])

0.898993755583708

In [9]:
sum(pca.explained_variance_ratio_[:4])

0.9277429537836405

Посчитав кумулятивные суммы, можно видеть, что 3 компоненты объясняют 89.9% дисперсии, а 4 - 92.77%

3. Примените построенное преобразование к исходным данным и возьмите значения первой компоненты.

In [10]:
first_comp = [x[0] for x in t]
first_comp

[-50.90240358101071,
 -52.84690918882607,
 -54.6144391707429,
 -52.60056627500222,
 -52.37012330050846,
 -54.65341197173625,
 -52.81257496142599,
 -53.651145700604935,
 -56.6927269799006,
 -54.40265505505307,
 -58.331645486603726,
 -61.71626498230254,
 -61.31940025440187,
 -53.792524454161025,
 -50.89263448325975,
 -49.63740755273961,
 -52.38761469790184,
 -47.47854294449206,
 -43.56033498710579,
 -42.559280462451305,
 -42.45027086574358,
 -42.27124797189937,
 -43.25272816148589,
 -40.05934481464537,
 -38.49726315245763,
 -39.06492262604911,
 -38.393043775493844,
 -38.400558325177684,
 -42.38616980919179,
 -39.83057417099061,
 -41.29548604098916,
 -40.43554957761839,
 -38.49607932244281,
 -42.96158052416134,
 -38.712521434784335,
 -38.833793314344675,
 -39.90606173856283,
 -37.58824522332664,
 -34.923567252430566,
 -32.96232693611815,
 -34.524617221835314,
 -36.4034594755264,
 -37.221946588331015,
 -32.958838614293455,
 -30.591593187058876,
 -29.63686919408284,
 -27.345138688018093,
 -

4. Загрузите информацию об индексе Доу-Джонса из файла djia_index.csv. Чему равна корреляция Пирсона между первой компонентой и индексом Доу-Джонса?

In [11]:
djia_index_data = pd.read_csv('_82b4a9f66c689b3d40dd25ebd761b07f_djia_index (1).csv')
djia_index_data = djia_index_data.drop('date', axis=1)
djia_index_data

,^DJI
0,15401.379883
1,15334.589844
2,15273.259766
3,15328.299805
4,15258.240234
...,...
369,17895.220703
370,17749.310547
371,17977.419922
372,17849.080078


In [12]:
pca.transform(djia_index_data)

array([[ 43305.26271674, -48792.50784745, -12341.49325408, ...,
        -23421.8388189 , -10739.03243356,    432.90196892],
       [ 43116.06620179, -48579.61569016, -12287.26050347, ...,
        -23319.79810691, -10692.10460666,    430.98945501],
       [ 42942.33614872, -48384.12706928, -12237.46117898, ...,
        -23226.09903262, -10649.01304199,    429.23328554],
       ...,
       [ 50602.42566296, -57003.59335351, -14433.2084123 , ...,
        -27357.46999444, -12549.00234608,    506.66614815],
       [ 50238.87667305, -56594.51219911, -14328.99792138, ...,
        -27161.3945129 , -12458.82857056,    502.99117295],
       [ 50882.21066278, -57318.41961869, -14513.40814602, ...,
        -27508.36844304, -12618.39951254,    509.49438578]])

In [13]:
np.corrcoef(first_comp, djia_index_data['^DJI'])

array([[1.        , 0.90965222],
       [0.90965222, 1.        ]])

5. Какая компания имеет наибольший вес в первой компоненте? Укажите ее название с большой буквы.

In [14]:
close_prices_data.columns[pca.components_[0].argmax()]

'V'